# Analysis of ODE-LSTM Results

This notebook creates the comparison between ODE-LSTM and MTS-LSTM from the paper. 
To reproduce the contents of this notebook, you need to download the models' predictions (or create them yourself) into the folder `BASE_DIR`

`README.md` contains information on where to obtain the required data.

In [1]:
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

from neuralhydrology.evaluation.metrics import calculate_metrics

BASE_DIR = Path('/home/mgauch/mts-lstm/results/odelstm')

metrics = ['NSE', 'MSE', 'RMSE', 'KGE', 'Alpha-NSE', 'Pearson-r', 'Beta-NSE', 'FHV', 'FMS', 'FLV', 'Peak-Timing']
basins = ['01022500', '02064000']

## Preparation
### Load predictions and metrics for each model ensemble

In [2]:
# (a) train on 1D+12H, evaluate on 1H (dividing 12H-predictions by 12)
# (b) train on 1H+3H, evaluate on 1D (aggregating every 8 3H-predictions)
# (c) train on 1H+1D, evaluate on 1H+1D
a_mtslstm, b_mtslstm = {}, {}
a_odelstm, b_odelstm = {}, {}
for b in basins:
    # MTS-LSTM predictions (single-basin)
    a_mtslstm[b] = pickle.load(open(BASE_DIR / f'ensemble_mtslstm_a_{b}.p', 'rb'))[b]
    b_mtslstm[b] = pickle.load(open(BASE_DIR / f'ensemble_mtslstm_b_{b}.p', 'rb'))[b]

    # ODE-LSTM (single-basin)
    a_odelstm[b] = pickle.load(open(BASE_DIR / f'ensemble_odelstm_a_{b}.p', 'rb'))[b]
    b_odelstm[b] = pickle.load(open(BASE_DIR / f'ensemble_odelstm_b_{b}.p', 'rb'))[b]

### (Dis-)aggregate MTS-LSTM predictions to missing timescales and calculate metrics

In [3]:
for basin in basins:
    a_mtslstm[basin]['1H']['xr'] = a_mtslstm[basin]['12H']['xr'].resample({'datetime': '1H'}).pad()
    b_mtslstm[basin]['1D']['xr'] = b_mtslstm[basin]['3H']['xr'].resample({'datetime': '1D'}).mean()
    a_mtslstm[basin]['1H']['xr']['qobs_mm_per_hour_obs'] = b_mtslstm[basin]['1H']['xr']['qobs_mm_per_hour_obs']
    b_mtslstm[basin]['1D']['xr']['qobs_mm_per_hour_obs'] = a_mtslstm[basin]['1D']['xr']['qobs_mm_per_hour_obs']

    a_metrics = calculate_metrics(a_mtslstm[basin]['1H']['xr']['qobs_mm_per_hour_obs'],
                                  a_mtslstm[basin]['1H']['xr']['qobs_mm_per_hour_sim'],
                                  metrics, resolution='1H')
    b_metrics = calculate_metrics(b_mtslstm[basin]['1D']['xr']['qobs_mm_per_hour_obs'],
                                  b_mtslstm[basin]['1D']['xr']['qobs_mm_per_hour_sim'],
                                  metrics, resolution='1D')
    for metric in metrics:
        a_mtslstm[basin]['1H'][metric + '_1H'] = a_metrics[metric]
        b_mtslstm[basin]['1D'][metric + '_1D'] = b_metrics[metric]

/home/mgauch/miniconda3/envs/pytorch/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/mgauch/miniconda3/envs/pytorch/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [4]:
def to_df(dict_mtslstm, dict_odelstm):
    return pd.DataFrame({'MTS-LSTM': dict_mtslstm, 'ODE-LSTM': dict_odelstm})

metric = 'NSE'

In [5]:
for basin in basins:
    print(basin)
    print(' (a) train on 1D+12H, evaluate on 1H:')
    display(to_df({f'{metric}_{f}': a_mtslstm[basin][f][f'{metric}_{f}'] for f in a_mtslstm[basin]},
                  {f'{metric}_{f}': a_odelstm[basin][f][f'{metric}_{f}'] for f in a_odelstm[basin]}))

    print(' (b) train on 1H+3H, evaluate on 1D:')
    display(to_df({f'{metric}_{f}': b_mtslstm[basin][f][f'{metric}_{f}'] for f in b_mtslstm[basin]},
                  {f'{metric}_{f}': b_odelstm[basin][f][f'{metric}_{f}'] for f in b_odelstm[basin]}))

01022500
 (a) train on 1D+12H, evaluate on 1H:


,MTS-LSTM,ODE-LSTM
NSE_1D,0.794056,0.779118
NSE_12H,0.798655,0.772819
NSE_1H,0.791883,0.766070


 (b) train on 1H+3H, evaluate on 1D:


,MTS-LSTM,ODE-LSTM
NSE_1H,0.806500,0.752522
NSE_3H,0.806255,0.756015
NSE_1D,0.808517,0.757462


02064000
 (a) train on 1D+12H, evaluate on 1H:


,MTS-LSTM,ODE-LSTM
NSE_1D,0.666761,0.712177
NSE_12H,0.701545,0.695307
NSE_1H,0.625982,0.613092


 (b) train on 1H+3H, evaluate on 1D:


,MTS-LSTM,ODE-LSTM
NSE_1H,0.681597,0.678696
NSE_3H,0.713100,0.660292
NSE_1D,0.729742,0.521488


## NSE of dis-aggregating observations

In [6]:
for basin in basins:
    print(basin)
    twelve_h = pickle.load(open(BASE_DIR / f'ensemble_mtslstm_a_{basin}.p', 'rb'))[basin]['12H']['xr']
    one_h = pickle.load(open(BASE_DIR / f'ensemble_mtslstm_b_{basin}.p', 'rb'))[basin]['1H']['xr']

    # dis-aggregate observations and calculate metrics
    disaggregated_obs = twelve_h.resample({'datetime': '1H'}).pad()
    disaggregated_obs['qobs_mm_per_hour_sim'] = disaggregated_obs['qobs_mm_per_hour_obs']
    disaggregated_obs['qobs_mm_per_hour_obs'] = one_h['qobs_mm_per_hour_obs']

    disaggregated_metrics = calculate_metrics(disaggregated_obs['qobs_mm_per_hour_obs'],
                                              disaggregated_obs['qobs_mm_per_hour_sim'],
                                              metrics, resolution='1H')

    display(disaggregated_metrics)

01022500


{'NSE': 0.9928403263911605,
 'MSE': 9.64436840149574e-05,
 'RMSE': 0.0098205745257066,
 'KGE': 0.9949282766526786,
 'Alpha-NSE': 0.9964137673377991,
 'Pearson-r': 0.9964137327203936,
 'Beta-NSE': -6.419479348096502e-08,
 'FHV': -0.4987846128642559,
 'FMS': -0.13019661441068692,
 'FLV': 4.98982127680642,
 'Peak-Timing': 4.758620689655173}

02064000


{'NSE': 0.8874261826276779,
 'MSE': 0.0003762401465792209,
 'RMSE': 0.019396910748343946,
 'KGE': 0.9180221965452662,
 'Alpha-NSE': 0.9420328736305237,
 'Pearson-r': 0.9420330049251419,
 'Beta-NSE': -6.44386261683394e-07,
 'FHV': -3.6287419497966766,
 'FMS': 1.3225497302728932,
 'FLV': 2.112789098930481,
 'Peak-Timing': 4.01010101010101}